I am using the hmdb51 dataset and a C3d model where the first n-1 layers are pre-trained and being used as a feature extractor. The pre-processing code to get the dataset as well as the pretrained C3D model were adapted from the following repo:
https://github.com/jfzhang95/pytorch-video-recognition

In [0]:
from torch.utils.data import DataLoader

train_data = VideoDataset(dataset='hmdb51', split='train', clip_len=8, preprocess=True)
train_loader = DataLoader(train_data, batch_size=100, shuffle=True, num_workers=4)

In [0]:
import torch
import torch.nn as nn

class C3D(nn.Module):


    def __init__(self, num_classes, pretrained=True):
        super(C3D, self).__init__()

        '''
        Pretrained feature extractor
        '''
        self.dropout = nn.Dropout(p=0.5)
        self.relu = nn.ReLU()
        self.conv1 = nn.Conv3d(3, 64, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.pool1 = nn.MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2))
        self.conv2 = nn.Conv3d(64, 128, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.pool2 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2))
        self.conv3a = nn.Conv3d(128, 256, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.conv3b = nn.Conv3d(256, 256, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.pool3 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2))
        self.conv4a = nn.Conv3d(256, 512, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.conv4b = nn.Conv3d(512, 512, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.pool4 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2))
        self.conv5a = nn.Conv3d(512, 512, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.conv5b = nn.Conv3d(512, 512, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.pool5 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=(0, 1, 1))
        self.fc6 = nn.Linear(8192, 4096)
        self.fc7 = nn.Linear(4096, 4096)

        '''
        Trained Layer
        '''
        self.fc8 = nn.Linear(4096, num_classes)

        if pretrained:
            self.__load_pretrained_weights()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool1(x)
        x = self.relu(self.conv2(x))
        x = self.pool2(x)
        x = self.relu(self.conv3a(x))
        x = self.relu(self.conv3b(x))
        x = self.pool3(x)
        x = self.relu(self.conv4a(x))
        x = self.relu(self.conv4b(x))
        x = self.pool4(x)
        x = self.relu(self.conv5a(x))
        x = self.relu(self.conv5b(x))
        x = self.pool5(x)
        x = x.view(-1, 8192)
        x = self.relu(self.fc6(x))
        x = self.dropout(x)
        x = self.relu(self.fc7(x))
        x = self.dropout(x)

        logits = self.fc8(x)
        return logits

    def __load_pretrained_weights(self):
        corresp_name = {
                        "features.0.weight": "conv1.weight",
                        "features.0.bias": "conv1.bias",
                        "features.3.weight": "conv2.weight",
                        "features.3.bias": "conv2.bias",
                        "features.6.weight": "conv3a.weight",
                        "features.6.bias": "conv3a.bias",
                        "features.8.weight": "conv3b.weight",
                        "features.8.bias": "conv3b.bias",
                        "features.11.weight": "conv4a.weight",
                        "features.11.bias": "conv4a.bias",
                        "features.13.weight": "conv4b.weight",
                        "features.13.bias": "conv4b.bias",
                        "features.16.weight": "conv5a.weight",
                        "features.16.bias": "conv5a.bias",
                        "features.18.weight": "conv5b.weight",
                        "features.18.bias": "conv5b.bias",
                        "classifier.0.weight": "fc6.weight",
                        "classifier.0.bias": "fc6.bias",
                        "classifier.3.weight": "fc7.weight",
                        "classifier.3.bias": "fc7.bias",
                        }

        p_dict = torch.load('c3d-pretrained.pth')
        s_dict = self.state_dict()
        for name in p_dict:
            if name not in corresp_name:
                continue
            s_dict[corresp_name[name]] = p_dict[name]
        self.load_state_dict(s_dict)

    def __init_weight(self):
        for m in self.modules():
            if isinstance(m, nn.Conv3d):
                torch.nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm3d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

**Training Code**

I trained the model for 20 epochs using Adam optimizer with a learning rate of 1e-3 and a batch size of 32.

In [0]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torch.autograd import Variable


num_epochs = 50
lr = 1e-3

model = C3D(num_classes=51, pretrained=True)


for param in model.parameters():
    param.requires_grad = False

model.fc8 = nn.Linear(4096, 51)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc8.parameters(), lr=3e-4)

model.cuda()
criterion.cuda()

train_dataloader = DataLoader(VideoDataset(dataset='hmdb51', split='train',clip_len=16), batch_size=32, shuffle=True, num_workers=4)
val_dataloader   = DataLoader(VideoDataset(dataset='hmdb51', split='val',  clip_len=16), batch_size=32, num_workers=4)
test_dataloader  = DataLoader(VideoDataset(dataset='hmdb51', split='test', clip_len=16), batch_size=32, num_workers=4)

training_loss = []
validation_loss = []

for z in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_dataloader):
        inputs = inputs.cuda()
        print(inputs.shape)
        labels = labels.cuda()
        print(labels.shape)

        outputs = model(inputs)

        probs = nn.Softmax(dim=1)(outputs)
        preds = torch.max(probs, 1)[1]
        loss = criterion(outputs, labels)

        print("Epoch:",z, "Loss:",loss.item())
        training_loss.append(loss.item())

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if i % 40 == 0:
            with torch.no_grad():
              total_val_loss = 0
              model.eval()

            m = 0
            for j, (inputs, labels) in enumerate(val_dataloader):
                inputs = inputs.cuda()
                print(inputs.shape)
                labels = labels.cuda()
                print(labels.shape)

                outputs = model(inputs)
                val_probs = nn.Softmax(dim=1)(outputs)
                val_preds = torch.max(val_probs, 1)[1]
                val_loss = criterion(outputs, labels)

                total_val_loss += val_loss.item()
                m += 1

            validation_loss.append(total_val_loss/m)
            model.train()

    torch.save(model.state_dict(), 'c3d_hdm51.pt')
    np.save('Training_Loss', np.array(training_loss))
    np.save('Val_loss', np.array(validation_loss))